In [86]:
# =============================================================================
# 셀 1: 라이브러리 및 초기 설정
# =============================================================================

import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

# 머신러닝 라이브러리
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import optuna
from optuna.samplers import TPESampler

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 메모리 관리
import gc
import psutil
import subprocess

print("필수 라이브러리 로드 완료")
print(f"XGBoost 버전: {xgb.__version__}")
print(f"Optuna 버전: {optuna.__version__}")


필수 라이브러리 로드 완료
XGBoost 버전: 3.0.4
Optuna 버전: 4.5.0


In [88]:
# =============================================================================
# 셀 2: 경로 설정 및 디렉토리 생성
# =============================================================================

# 경로 설정
data_dir = "/workspace/AI모델/projects/coin/data/v01/crypto_xgboost"  # 전처리된 데이터 경로
output_dir = "/workspace/AI모델/projects/coin/models/v01/crypto_xgboost"  # 모델 저장 경로

# 디렉토리 생성
os.makedirs(output_dir, exist_ok=True)

# 설정값
N_FOLDS = 5     
N_TRIALS = 20   # 20번 시도 (암호화폐는 더 많은 시도 필요)
RANDOM_STATE = 42

# GPU 최적화 설정
USE_GPU = True
GPU_DEVICE = 'cuda:0'
GPU_BATCH_SIZE = 10000
PARALLEL_JOBS = 1      # GPU 사용시에는 1 또는 2가 최적

# 메모리 모니터링 활성화
MEMORY_MONITORING = True

print(f"데이터 디렉토리: {data_dir}")
print(f"출력 디렉토리: {output_dir}")
print(f"K-Fold: {N_FOLDS}폴드, Optuna 시도: {N_TRIALS}회")
print(f"GPU 사용: {USE_GPU}")

# 시스템 메모리 확인
total_ram = psutil.virtual_memory().total / (1024**3)
available_ram = psutil.virtual_memory().available / (1024**3)
print(f"시스템 RAM: {total_ram:.1f}GB (사용가능: {available_ram:.1f}GB)")

데이터 디렉토리: /workspace/AI모델/projects/coin/data/v01/crypto_xgboost
출력 디렉토리: /workspace/AI모델/projects/coin/models/v01/crypto_xgboost
K-Fold: 5폴드, Optuna 시도: 20회
GPU 사용: True
시스템 RAM: 47.0GB (사용가능: 38.4GB)


# 피처엔지니어링

In [90]:
# =============================================================================
# 셀 3: 데이터 로드 및 전처리
# =============================================================================

def load_and_prepare_crypto_data(data_dir):
    """암호화폐 데이터 로드 및 전처리"""
    print("암호화폐 데이터 로드 시작...")
    
    # 1. training_features.txt 파일 읽기
    feature_file = os.path.join(data_dir, 'training_features.txt')
    
    if os.path.exists(feature_file):
        with open(feature_file, 'r') as f:
            base_features = [line.strip() for line in f.readlines() if line.strip()]
        print(f"training_features.txt에서 {len(base_features)}개 기본 피처 로드")
    else:
        print(f"⚠️ training_features.txt 파일이 없습니다: {feature_file}")
        base_features = None
    
    # 2. 데이터 로드
    train_df = pd.read_csv(os.path.join(data_dir, 'train_data.csv'))
    val_df = pd.read_csv(os.path.join(data_dir, 'val_data.csv'))
    test_df = pd.read_csv(os.path.join(data_dir, 'test_data.csv'))
    
    print(f"데이터 로드 완료:")
    print(f"  훈련: {train_df.shape}")
    print(f"  검증: {val_df.shape}")
    print(f"  테스트: {test_df.shape}")
        
    # 3. 암호화폐 특성에 맞는 추가 제외 피처들
    crypto_exclude = [
        # 원본 OHLCV 데이터
        'Open', 'High', 'Low', 'Close', 'Volume',
        
        # 라벨 관련
        'Future_Label', 'Label', 'Optimized_Label', 'Future_Label_Name',
        'Label_Name', 'Optimized_Label_Name',
        
        # 데이터 누수 방지 - 모든 Future 관련 컬럼들
        'Future_Return_1d', 'Future_Return_3d', 'Future_Return_7d', 
        'Future_Return_14d', 'Future_Return_30d', 'Future_Return_90d',
        'Future_Label_1d', 'Future_Label_3d', 'Future_Label_7d',
        'Future_Label_14d', 'Future_Label_30d', 'Future_Label_90d',
        'Technical_Score', 'Final_Score',


        # 메타 데이터
        'Date', 'Symbol',
        
        # 완전 중복 지표들 (상관계수 1.0)
        'SMI_ema', 'SMI', 'SMI_normalized', # SMI_Signal와 높은 상관관계
        
        # 높은 상관관계 피처들 (암호화폐 특성상 제거)
        'MACD_7_14_signal_distance', 'MACD_14_30_signal_distance',
        
        # VIF가 높은 피처들
        'realized_volatility_5', 'realized_volatility_20',
        
        # 시간 관련 중복
        'quarter', 'dow_cos',  # 다른 시간 피처로 충분히 표현됨
        
        # 암호화폐에서 효과가 낮은 지표들
        'year',  # 암호화폐는 짧은 역사로 year 피처 의미 제한적
    ]
    
    # 4. 최종 피쳐 결정
    if base_features is not None:
        # training_features.txt에서 제외 항목을 빼기
        feature_columns = [f for f in base_features if f not in crypto_exclude]
        print(f"training_features.txt 기반 피처: {len(base_features)}개")
        print(f"피처 제외 항목: {len(crypto_exclude)}개")
    else:
        # training_features.txt가 없으면 전체 컬럼에서 제외
        feature_columns = [col for col in train_df.columns if col not in crypto_exclude]
    
    # 5. 실제 존재하는 피쳐만 필터링
    feature_columns = [col for col in feature_columns if col in train_df.columns]
    
    # Future_Label 제거 (타겟이므로)
    if 'Future_Label' in feature_columns:
        feature_columns.remove('Future_Label')
    
    print(f"최종 피쳐 수: {len(feature_columns)}")
    
    # 6. Future_Label 확인
    if 'Future_Label' not in train_df.columns:
        raise ValueError("Future_Label 컬럼이 없습니다!")
    
    # 7. 훈련+검증 데이터 합치기
    train_val_df = pd.concat([train_df, val_df], ignore_index=True)
    
    # 8. 피쳐와 타겟 분리
    X_train_val = train_val_df[feature_columns].fillna(0)
    y_train_val = train_val_df['Future_Label']
    
    X_test = test_df[feature_columns].fillna(0)
    y_test = test_df['Future_Label']
    
    # 9. 무한값 처리
    X_train_val = X_train_val.replace([np.inf, -np.inf], 0)
    X_test = X_test.replace([np.inf, -np.inf], 0)
    
    print(f"데이터 전처리 완료:")
    print(f"  X_train_val: {X_train_val.shape}")
    print(f"  X_test: {X_test.shape}")
    
    # 10. 클래스 분포 확인 (암호화폐 라벨)
    print(f"클래스 분포:")
    label_names = {0: 'Strong_Sell', 1: 'Hold', 2: 'Strong_Buy'}
    
    for name, y in [("훈련+검증", y_train_val), ("테스트", y_test)]:
        counts = y.value_counts().sort_index()
        pcts = (counts / len(y) * 100).round(1)
        
        print(f"  {name}:")
        for label, count in counts.items():
            pct = pcts[label]
            label_name = label_names.get(int(label), f'Label_{int(label)}')
            print(f"    {int(label)} ({label_name}): {count:,}개 ({pct}%)")
    
    # 11. 클래스 균형도
    class_ratio = y_train_val.value_counts().min() / y_train_val.value_counts().max()
    print(f"클래스 균형도: {class_ratio:.3f} (1.0에 가까울수록 균형)")
    
    # 12. 데이터 검증
    print(f"데이터 검증:")
    print(f"  결측치 - X_train_val: {X_train_val.isnull().sum().sum()}")
    print(f"  결측치 - X_test: {X_test.isnull().sum().sum()}")
    print(f"  무한값 - X_train_val: {np.isinf(X_train_val.values).sum()}")
    print(f"  무한값 - X_test: {np.isinf(X_test.values).sum()}")
    
    return X_train_val, X_test, y_train_val, y_test, feature_columns

# 데이터 로드 실행
X_train_val, X_test, y_train_val, y_test, feature_columns = load_and_prepare_crypto_data(data_dir)

print(f"데이터 준비 완료!")
print(f"사용할 피쳐 수: {len(feature_columns)}")
print(f"주요 피쳐들:")
for i, feat in enumerate(feature_columns[:15]):
    print(f"  {i+1:2d}. {feat}")
if len(feature_columns) > 15:
    print(f"  ... 외 {len(feature_columns)-15}개")

암호화폐 데이터 로드 시작...
training_features.txt에서 46개 기본 피처 로드
데이터 로드 완료:
  훈련: (65536, 60)
  검증: (14048, 60)
  테스트: (14067, 60)
training_features.txt 기반 피처: 46개
피처 제외 항목: 37개
최종 피쳐 수: 28
데이터 전처리 완료:
  X_train_val: (79584, 28)
  X_test: (14067, 28)
클래스 분포:
  훈련+검증:
    0 (Strong_Sell): 9,897개 (12.4%)
    0 (Strong_Sell): 3개 (0.0%)
    1 (Hold): 59,765개 (75.1%)
    1 (Hold): 3개 (0.0%)
    2 (Strong_Buy): 9,916개 (12.5%)
  테스트:
    0 (Strong_Sell): 1,580개 (11.2%)
    1 (Hold): 10,891개 (77.4%)
    2 (Strong_Buy): 1,596개 (11.3%)
클래스 균형도: 0.000 (1.0에 가까울수록 균형)
데이터 검증:
  결측치 - X_train_val: 0
  결측치 - X_test: 0
  무한값 - X_train_val: 0
  무한값 - X_test: 0
데이터 준비 완료!
사용할 피쳐 수: 28
주요 피쳐들:
   1. Stoch_14_K_above_D
   2. Stoch_6_K_above_D
   3. CCI_4_overbought
   4. Formula3_Signal
   5. Stoch_1_K_above_D
   6. Stoch_4_K_above_D
   7. is_quarter_start
   8. high_volatility_regime
   9. is_month_end
  10. Stoch_3_K_above_D
  11. Stoch_K_3
  12. is_month_start
  13. Momentum_Signal
  14. day_of_week
  15. Volat

In [91]:
# =============================================================================
# 셀 4: K-Fold 교차검증 설정 및 모니터링 함수
# =============================================================================

# K-Fold 설정
kfold = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)

# 클래스 가중치 계산 함수
def calculate_class_weights(y):
    """클래스 가중치 계산 (암호화폐 불균형 해결)"""
    class_counts = y.value_counts().sort_index()
    total_samples = len(y)
    class_weights = {i: total_samples / (len(class_counts) * count) 
                    for i, count in class_counts.items()}
    return class_weights

# 클래스 가중치 확인
class_weights = calculate_class_weights(y_train_val)
print(f"클래스 가중치: {class_weights}")

# 모니터링 함수들
def check_gpu_memory():
    """GPU 메모리 사용률 체크"""
    try:
        result = subprocess.run(['nvidia-smi', '--query-gpu=utilization.gpu,memory.used,memory.total', 
                               '--format=csv,noheader,nounits'], 
                              capture_output=True, text=True)
        if result.returncode == 0:
            gpu_util, mem_used, mem_total = result.stdout.strip().split(', ')
            mem_percent = int(mem_used) / int(mem_total) * 100
            return f"GPU: {gpu_util}% | VRAM: {mem_percent:.1f}%"
        else:
            return "GPU 정보 없음"
    except:
        return "nvidia-smi 없음"

def check_system_memory():
    """시스템 메모리 사용률 체크"""
    mem = psutil.virtual_memory()
    return f"RAM: {mem.percent:.1f}%"

def clear_memory():
    """메모리 정리"""
    gc.collect()
    if USE_GPU:
        try:
            import torch
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
        except:
            pass

# GPU 메모리 최적화
def optimize_gpu_memory():
    """GPU 메모리 사용량 최적화"""
    if USE_GPU:
        os.environ['CUDA_VISIBLE_DEVICES'] = '0'
        os.environ['XGB_CUDA_MEMORY_POOL'] = '8192'  # 8GB GPU 메모리 풀
        try:
            import cupy
            mempool = cupy.get_default_memory_pool()
            mempool.set_limit(size=8 * 1024**3)  # 8GB 제한
            print("GPU 메모리 풀 설정: 8GB")
        except:
            print("CuPy 없음 - XGBoost 내장 GPU 메모리 관리 사용")

optimize_gpu_memory()
print(f"K-Fold 교차검증 설정 완료 ({N_FOLDS}폴드)")
print("모니터링 함수 설정 완료")

클래스 가중치: {0.0: 1.6082449227038496, 0.8780000000000001: 5305.6, 1.0: 0.26632309880364763, 1.455000000000382: 5305.6, 2.0: 1.6051633723275514}
CuPy 없음 - XGBoost 내장 GPU 메모리 관리 사용
K-Fold 교차검증 설정 완료 (5폴드)
모니터링 함수 설정 완료


In [96]:
# =============================================================================
# 데이터 전처리 및 검증 (Optuna 실행 전에 먼저 실행)
# =============================================================================

print("📊 라벨 데이터 검증 및 전처리...")

# 1. y_train_val 데이터 타입 및 분포 확인
print(f"y_train_val 원본 정보:")
print(f"  데이터 타입: {y_train_val.dtype}")
print(f"  결측치 수: {y_train_val.isnull().sum()}")
print(f"  고유값: {sorted(y_train_val.unique())}")
print(f"  값 분포:")
print(y_train_val.value_counts().sort_index())

# 2. 라벨 데이터 정리
# NaN 제거 및 정수형 변환
y_train_val_clean = y_train_val.dropna()
X_train_val_clean = X_train_val.loc[y_train_val_clean.index]

# 정수형으로 변환
y_train_val_clean = y_train_val_clean.astype(int)

# 유효한 라벨만 유지 (0, 1, 2)
valid_mask = y_train_val_clean.isin([0, 1, 2])
y_train_val_final = y_train_val_clean[valid_mask]
X_train_val_final = X_train_val_clean.loc[y_train_val_final.index]

print(f"\n정리 후 데이터:")
print(f"  X_train_val: {X_train_val_final.shape}")
print(f"  y_train_val: {y_train_val_final.shape}")
print(f"  데이터 타입: {y_train_val_final.dtype}")
print(f"  라벨 분포:")
print(y_train_val_final.value_counts().sort_index())

# 클래스별 최소 샘플 수 확인
min_class_size = y_train_val_final.value_counts().min()
print(f"  최소 클래스 크기: {min_class_size}")

# K-Fold 개수 조정 (최소 클래스 크기에 맞춰)
optimal_k_folds = min(5, min_class_size // 2)  # 각 클래스가 최소 2개씩은 있도록
print(f"  최적 K-Fold 수: {optimal_k_folds}")

📊 라벨 데이터 검증 및 전처리...
y_train_val 원본 정보:
  데이터 타입: float64
  결측치 수: 0
  고유값: [0.0, 0.8780000000000001, 1.0, 1.455000000000382, 2.0]
  값 분포:
Future_Label
0.000     9897
0.878        3
1.000    59765
1.455        3
2.000     9916
Name: count, dtype: int64

정리 후 데이터:
  X_train_val: (79584, 28)
  y_train_val: (79584,)
  데이터 타입: int64
  라벨 분포:
Future_Label
0     9900
1    59768
2     9916
Name: count, dtype: int64
  최소 클래스 크기: 9900
  최적 K-Fold 수: 5


In [97]:
# =============================================================================
# 셀 5: 수정된 Optuna 목적 함수 정의
# =============================================================================

def objective_function(trial):
    """수정된 암호화폐용 Optuna 목적 함수"""
    trial_num = trial.number + 1
    
    # 암호화폐에 특화된 파라미터 범위
    params = {
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',
        'max_depth': trial.suggest_int('max_depth', 4, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.25),
        'n_estimators': trial.suggest_int('n_estimators', 200, 800),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.6, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 2),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 3),
        'max_leaves': trial.suggest_int('max_leaves', 50, 300),
        'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
        'early_stopping_rounds': 50,
        'random_state': RANDOM_STATE,
        'n_jobs': PARALLEL_JOBS,
        'verbosity': 0
    }
    
    # GPU 최적화 설정
    if USE_GPU:
        params['tree_method'] = 'gpu_hist'
        params['gpu_id'] = 0
        params['predictor'] = 'gpu_predictor'
        params['max_bin'] = trial.suggest_int('max_bin', 256, 1024)
        params['single_precision_histogram'] = True
    
    # 수정된 K-Fold 교차검증 (정리된 데이터 사용)
    kfold_dynamic = StratifiedKFold(
        n_splits=optimal_k_folds, 
        shuffle=True, 
        random_state=RANDOM_STATE
    )
    
    cv_scores = []
    
    try:
        for fold_num, (train_idx, val_idx) in enumerate(kfold_dynamic.split(X_train_val_final, y_train_val_final), 1):
            X_train_fold = X_train_val_final.iloc[train_idx]
            X_val_fold = X_train_val_final.iloc[val_idx]
            y_train_fold = y_train_val_final.iloc[train_idx]
            y_val_fold = y_train_val_final.iloc[val_idx]
            
            # 클래스 가중치 계산 (정수형 라벨 사용)
            fold_class_weights = calculate_class_weights(y_train_fold)
            sample_weights = np.array([fold_class_weights[label] for label in y_train_fold])
            
            # 모델 학습
            model = xgb.XGBClassifier(**params)
            model.fit(
                X_train_fold, y_train_fold,
                sample_weight=sample_weights,
                eval_set=[(X_val_fold, y_val_fold)],
                verbose=False
            )
            
            # 예측 및 평가
            val_pred = model.predict(X_val_fold)
            f1 = f1_score(y_val_fold, val_pred, average='macro')
            cv_scores.append(f1)
            
            # 폴드 완료 후 정리
            del model, val_pred
            clear_memory()
    
    except Exception as e:
        print(f"Trial {trial_num} 오류: {e}")
        return 0.0  # 실패한 경우 낮은 점수 반환
    
    if not cv_scores:  # 빈 리스트인 경우
        return 0.0
    
    mean_cv_score = np.mean(cv_scores)
    std_cv_score = np.std(cv_scores)
    
    print(f"Trial {trial_num} 결과:")
    print(f"  평균 CV F1-Score: {mean_cv_score:.4f} ± {std_cv_score:.4f}")
    print(f"  개별 폴드 점수: {[f'{score:.4f}' for score in cv_scores]}")
    
    # Trial 완료 후 메모리 정리
    clear_memory()
    
    return mean_cv_score

print("✅ 수정된 Optuna 목적 함수 정의 완료")

✅ 수정된 Optuna 목적 함수 정의 완료


# 하이퍼파라미터 튜닝

In [98]:
# =============================================================================
# 셀 6: 하이퍼파라미터 최적화 실행
# =============================================================================

print("🚀 하이퍼파라미터 최적화 시작...")
print(f"최적화 시도 횟수: {N_TRIALS}회")
print(f"K-Fold 수: {optimal_k_folds}")
print(f"사용 데이터: {X_train_val_final.shape}")
print(f"GPU 사용: {USE_GPU}")

# 최종 데이터 검증
print(f"\n최종 라벨 분포:")
for label, count in y_train_val_final.value_counts().sort_index().items():
    label_name = {0: 'Sell', 1: 'Hold', 2: 'Buy'}[label]
    pct = count / len(y_train_val_final) * 100
    print(f"  {label} ({label_name}): {count:,}개 ({pct:.1f}%)")

# Optuna Study 생성
study = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=RANDOM_STATE)
)

# 최적화 실행
def print_best_callback(study, trial):
    """각 trial 후 현재까지 최고 성능 출력"""
    if trial.number > 0:
        print(f"\n현재까지 최고 성능:")
        print(f"  Best F1-Score: {study.best_value:.4f}")
        print(f"  Best Trial: {study.best_trial.number + 1}")
        print(f"  남은 시도: {N_TRIALS - trial.number - 1}회")
        print("-" * 50)

try:
    study.optimize(objective_function, n_trials=N_TRIALS, callbacks=[print_best_callback])
    
    # 결과 저장
    best_params = study.best_params
    best_cv_score = study.best_value
    
    print(f"{'='*60}")
    print(f"🎉 하이퍼파라미터 최적화 완료!")
    print(f"{'='*60}")
    print(f"최고 CV F1-Score: {best_cv_score:.4f}")
    print(f"최고 성능을 낸 Trial: {study.best_trial.number + 1}")
    print(f"사용된 K-Fold 수: {optimal_k_folds}")
    print(f"\n최적 파라미터:")
    for key, value in best_params.items():
        print(f"  {key}: {value}")
    
    # Top 5 시도 결과 출력
    print(f"\nTop 5 시도 결과:")
    sorted_trials = sorted(study.trials, key=lambda x: x.value if x.value else 0, reverse=True)[:5]
    for i, trial in enumerate(sorted_trials, 1):
        if trial.value:
            print(f"  {i}. Trial {trial.number + 1}: F1-Score {trial.value:.4f}")

except Exception as e:
    print(f"❌ 최적화 중 오류 발생: {e}")
    print("데이터를 다시 확인해주세요.")

print(f"\n✅ 최적화 프로세스 완료")

# =============================================================================
# 추가: 정리된 데이터를 전역 변수로 업데이트 (다음 셀에서 사용하기 위해)
# =============================================================================

# 다음 셀들에서 사용할 수 있도록 정리된 데이터로 업데이트
X_train_val = X_train_val_final
y_train_val = y_train_val_final

print(f"\n📊 전역 변수 업데이트 완료:")
print(f"  X_train_val: {X_train_val.shape}")
print(f"  y_train_val: {y_train_val.shape}")
print(f"  라벨 타입: {y_train_val.dtype}")

[I 2025-09-28 03:21:48,462] A new study created in memory with name: no-name-4a3c7eb9-085a-40d1-9c4f-b7f11c3afae1


🚀 하이퍼파라미터 최적화 시작...
최적화 시도 횟수: 20회
K-Fold 수: 5
사용 데이터: (79584, 28)
GPU 사용: True

최종 라벨 분포:
  0 (Sell): 9,900개 (12.4%)
  1 (Hold): 59,768개 (75.1%)
  2 (Buy): 9,916개 (12.5%)


[I 2025-09-28 03:22:27,197] Trial 0 finished with value: 0.4317890839149661 and parameters: {'max_depth': 7, 'learning_rate': 0.23817143353837988, 'n_estimators': 639, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'colsample_bylevel': 0.662397808134481, 'colsample_bynode': 0.6232334448672797, 'min_child_weight': 9, 'gamma': 3.005575058716044, 'reg_alpha': 1.416145155592091, 'reg_lambda': 0.1596950334578271, 'max_leaves': 293, 'grow_policy': 'depthwise', 'max_bin': 395}. Best is trial 0 with value: 0.4317890839149661.


Trial 1 결과:
  평균 CV F1-Score: 0.4318 ± 0.0018
  개별 폴드 점수: ['0.4324', '0.4313', '0.4286', '0.4329', '0.4337']


[I 2025-09-28 03:24:25,287] Trial 1 finished with value: 0.43864761528016183 and parameters: {'max_depth': 5, 'learning_rate': 0.08301813831028905, 'n_estimators': 515, 'subsample': 0.7727780074568463, 'colsample_bytree': 0.7164916560792167, 'colsample_bylevel': 0.8447411578889518, 'colsample_bynode': 0.6557975442608167, 'min_child_weight': 3, 'gamma': 1.8318092164684585, 'reg_alpha': 0.9121399684340719, 'reg_lambda': 2.3770102880397395, 'max_leaves': 100, 'grow_policy': 'lossguide', 'max_bin': 291}. Best is trial 1 with value: 0.43864761528016183.


...

Trial 18 결과:
  평균 CV F1-Score: 0.5128 ± 0.0035
  개별 폴드 점수: ['0.5077', '0.5150', '0.5104', '0.5132', '0.5178']

현재까지 최고 성능:
  Best F1-Score: 0.5392
  Best Trial: 9
  남은 시도: 2회
--------------------------------------------------


[I 2025-09-28 05:14:41,596] Trial 18 finished with value: 0.42962144432575167 and parameters: {'max_depth': 8, 'learning_rate': 0.16933894770312002, 'n_estimators': 411, 'subsample': 0.8759710886560624, 'colsample_bytree': 0.7596641733462661, 'colsample_bylevel': 0.8396815651256182, 'colsample_bynode': 0.8988350397643667, 'min_child_weight': 8, 'gamma': 4.996467385418862, 'reg_alpha': 0.6782380447304142, 'reg_lambda': 0.6996355916121462, 'max_leaves': 200, 'grow_policy': 'depthwise', 'max_bin': 555}. Best is trial 8 with value: 0.5391506283854346.


Trial 19 결과:
  평균 CV F1-Score: 0.4296 ± 0.0016
  개별 폴드 점수: ['0.4313', '0.4311', '0.4295', '0.4269', '0.4293']

현재까지 최고 성능:
  Best F1-Score: 0.5392
  Best Trial: 9
  남은 시도: 1회
--------------------------------------------------


[I 2025-09-28 05:16:58,468] Trial 19 finished with value: 0.4664367544268188 and parameters: {'max_depth': 10, 'learning_rate': 0.24985923858104447, 'n_estimators': 781, 'subsample': 0.6512724702232536, 'colsample_bytree': 0.6879350499169388, 'colsample_bylevel': 0.8658452477859441, 'colsample_bynode': 0.8338171176295528, 'min_child_weight': 4, 'gamma': 2.2653392633121174, 'reg_alpha': 0.35302365926526263, 'reg_lambda': 1.833883131947237, 'max_leaves': 175, 'grow_policy': 'lossguide', 'max_bin': 909}. Best is trial 8 with value: 0.5391506283854346.


Trial 20 결과:
  평균 CV F1-Score: 0.4664 ± 0.0021
  개별 폴드 점수: ['0.4648', '0.4637', '0.4661', '0.4682', '0.4693']

현재까지 최고 성능:
  Best F1-Score: 0.5392
  Best Trial: 9
  남은 시도: 0회
--------------------------------------------------
🎉 하이퍼파라미터 최적화 완료!
최고 CV F1-Score: 0.5392
최고 성능을 낸 Trial: 9
사용된 K-Fold 수: 5

최적 파라미터:
  max_depth: 11
  learning_rate: 0.22506191198163839
  n_estimators: 391
  subsample: 0.6440207698110707
  colsample_bytree: 0.6911740650167767
  colsample_bylevel: 0.7708431154505025
  colsample_bynode: 0.9272059063689972
  min_child_weight: 9
  gamma: 0.03476065265595352
  reg_alpha: 1.0214946051551315
  reg_lambda: 1.310491909131459
  max_leaves: 105
  grow_policy: lossguide
  max_bin: 981

Top 5 시도 결과:
  1. Trial 9: F1-Score 0.5392
  2. Trial 6: F1-Score 0.5372
  3. Trial 15: F1-Score 0.5300
  4. Trial 16: F1-Score 0.5222
  5. Trial 12: F1-Score 0.5186

✅ 최적화 프로세스 완료

📊 전역 변수 업데이트 완료:
  X_train_val: (79584, 28)
  y_train_val: (79584,)
  라벨 타입: int64


In [99]:
# =============================================================================
# 셀 7: 최종 모델 학습
# =============================================================================

print("최종 모델 학습 시작...")
print(f"학습 전 상태 | {check_gpu_memory()} | {check_system_memory()}")

# 메모리 정리
clear_memory()

# 최적 파라미터에 기본 파라미터 추가
final_params = {
    **best_params,
    'objective': 'multi:softprob',
    'num_class': 3,
    'eval_metric': 'mlogloss',
    'random_state': RANDOM_STATE,
    'n_jobs': PARALLEL_JOBS,
    'verbosity': 1
}

# GPU 설정
if USE_GPU:
    final_params['tree_method'] = 'gpu_hist'
    final_params['gpu_id'] = 0
    final_params['predictor'] = 'gpu_predictor'
    final_params['single_precision_histogram'] = True
    print("GPU 가속 사용")

print(f"최종 모델 파라미터:")
for key, value in final_params.items():
    print(f"  {key}: {value}")

# 샘플 가중치 계산
sample_weights = np.array([class_weights[label] for label in y_train_val])
print(f"클래스 가중치 적용: {class_weights}")
print(f"학습 준비 완료 | {check_gpu_memory()} | {check_system_memory()}")

# 최종 모델 학습
print(f"최종 모델 학습 진행...")
final_model = xgb.XGBClassifier(**final_params)

final_model.fit(
    X_train_val, y_train_val,
    sample_weight=sample_weights,
    eval_set=[(X_test, y_test)],
    verbose=True
)

print(f"최종 모델 학습 완료!")


최종 모델 학습 시작...
학습 전 상태 | GPU: 30% | VRAM: 16.0% | RAM: 18.8%
GPU 가속 사용
최종 모델 파라미터:
  max_depth: 11
  learning_rate: 0.22506191198163839
  n_estimators: 391
  subsample: 0.6440207698110707
  colsample_bytree: 0.6911740650167767
  colsample_bylevel: 0.7708431154505025
  colsample_bynode: 0.9272059063689972
  min_child_weight: 9
  gamma: 0.03476065265595352
  reg_alpha: 1.0214946051551315
  reg_lambda: 1.310491909131459
  max_leaves: 105
  grow_policy: lossguide
  max_bin: 981
  objective: multi:softprob
  num_class: 3
  eval_metric: mlogloss
  random_state: 42
  n_jobs: 1
  verbosity: 1
  tree_method: gpu_hist
  gpu_id: 0
  predictor: gpu_predictor
  single_precision_histogram: True
클래스 가중치 적용: {0.0: 1.6082449227038496, 0.8780000000000001: 5305.6, 1.0: 0.26632309880364763, 1.455000000000382: 5305.6, 2.0: 1.6051633723275514}
학습 준비 완료 | GPU: 30% | VRAM: 16.2% | RAM: 18.8%
최종 모델 학습 진행...
[0]	validation_0-mlogloss:1.06322
[1]	validation_0-mlogloss:1.04023
[2]	validation_0-mlogloss:1.01394
[3

# 모델 Test & Save

In [100]:
# =============================================================================
# 셀 8: 모델 평가 및 저장
# =============================================================================
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report, 
    confusion_matrix, roc_auc_score
)

def evaluate_model_comprehensive(model, X_train_val, X_test, y_train_val, y_test, cv_score):
    """포괄적인 모델 평가"""
    print("포괄적인 모델 평가 시작...")
    
    # 예측 수행
    train_pred = model.predict(X_train_val)
    test_pred = model.predict(X_test)
    train_pred_proba = model.predict_proba(X_train_val)
    test_pred_proba = model.predict_proba(X_test)
    
    # Train 성능
    train_f1 = f1_score(y_train_val, train_pred, average='macro')
    train_accuracy = accuracy_score(y_train_val, train_pred)
    
    # Test 성능
    test_f1 = f1_score(y_test, test_pred, average='macro')
    test_accuracy = accuracy_score(y_test, test_pred)
    
    # 과적합 분석
    overfitting_gap = train_f1 - test_f1
    cv_test_gap = abs(cv_score - test_f1)
    
    print("="*60)
    print("성능 요약")
    print("="*60)
    print(f"{'':15s} {'Accuracy':>10s} {'F1-Macro':>10s}")
    print("-"*40)
    print(f"{'Train':15s} {train_accuracy:10.4f} {train_f1:10.4f}")
    print(f"{'Test':15s} {test_accuracy:10.4f} {test_f1:10.4f}")
    print(f"{'CV (Best)':15s} {'':10s} {cv_score:10.4f}")
    
    print("\n과적합 분석:")
    print(f"Train-Test 차이: {overfitting_gap:.4f} ({overfitting_gap*100:.1f}%p)")
    print(f"CV-Test 차이: {cv_test_gap:.4f}")
    
    # if overfitting_gap < 0.02:
    #     print("과적합 수준: 낮음 (양호)")
    # elif overfitting_gap < 0.05:
    #     print("과적합 수준: 보통 (주의)")
    # else:
    #     print("과적합 수준: 높음 (개선 필요)")
    
    # 상세 분류 보고서
    print("\n상세 분류 보고서 (Test Set):")
    label_names = ['Strong_Sell', 'Hold', 'Strong_Buy']
    print(classification_report(y_test, test_pred, target_names=label_names))
    
    # 혼동행렬
    cm = confusion_matrix(y_test, test_pred)
    print("혼동행렬:")
    print(f"{'':10s} {'Sell':>6s} {'Hold':>6s} {'Buy':>6s}")
    for i, label in enumerate(label_names):
        print(f"{label:10s}", end="")
        for j in range(3):
            print(f"{cm[i,j]:6d}", end="")
        print()
    
    return {
        'train_f1': train_f1,
        'test_f1': test_f1,
        'train_accuracy': train_accuracy,
        'test_accuracy': test_accuracy,
        'cv_score': cv_score,
        'overfitting_gap': overfitting_gap,
        'cv_test_gap': cv_test_gap,
        'confusion_matrix': cm,
        'test_predictions': test_pred,
        'test_probabilities': test_pred_proba
    }

# 평가 실행
evaluation_results = evaluate_model_comprehensive(
    final_model, X_train_val, X_test, y_train_val, y_test, best_cv_score
)

# 피쳐 중요도 계산
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n피쳐 중요도 Top 20:")
for i, row in feature_importance.head(20).iterrows():
    print(f"  {i+1:2d}. {row['feature']:30s}: {row['importance']:.6f}")

# 모델 및 결과 저장
import joblib
import json

# 모델 저장
model_path = os.path.join(output_dir, 'crypto_xgboost_final_model.pkl')
joblib.dump(final_model, model_path)

# 피쳐 리스트 저장 (모델이 요구하는 피쳐 순서)
with open(os.path.join(output_dir, 'required_features.txt'), 'w') as f:
    for feature in feature_columns:
        f.write(f"{feature}\n")

# 피쳐 중요도 저장
feature_importance.to_csv(os.path.join(output_dir, 'feature_importance.csv'), index=False)

# 최적 파라미터 저장
with open(os.path.join(output_dir, 'best_params.json'), 'w') as f:
    json.dump(best_params, f, indent=2)

# 평가 결과 저장
label_names = ['Strong_Sell', 'Hold', 'Strong_Buy']
final_evaluation_results = {
    'performance': {
        'train_f1': evaluation_results['train_f1'],
        'test_f1': evaluation_results['test_f1'],
        'train_accuracy': evaluation_results['train_accuracy'],
        'test_accuracy': evaluation_results['test_accuracy'],
        'cv_score': evaluation_results['cv_score'],
        'overfitting_gap': evaluation_results['overfitting_gap']
    },
    'confusion_matrix': evaluation_results['confusion_matrix'].tolist(),
    'classification_report': classification_report(y_test, evaluation_results['test_predictions'], 
                                                  target_names=label_names, output_dict=True),
    'feature_count': len(feature_columns),
    'data_shape': {
        'train_val': X_train_val.shape,
        'test': X_test.shape
    }
}

with open(os.path.join(output_dir, 'evaluation_results.json'), 'w') as f:
    json.dump(final_evaluation_results, f, indent=2)

print(f"\n모델 및 결과 저장 완료: {output_dir}")
print(f"저장된 파일들:")
print(f"  - crypto_xgboost_final_model.pkl (모델)")
print(f"  - required_features.txt (피쳐 순서)")
print(f"  - feature_importance.csv (피쳐 중요도)")
print(f"  - best_params.json (최적 파라미터)")
print(f"  - evaluation_results.json (평가 결과)")

포괄적인 모델 평가 시작...
성능 요약
                  Accuracy   F1-Macro
----------------------------------------
Train               0.8471     0.8050
Test                0.6436     0.5039
CV (Best)                      0.5392

과적합 분석:
Train-Test 차이: 0.3010 (30.1%p)
CV-Test 차이: 0.0352

상세 분류 보고서 (Test Set):
              precision    recall  f1-score   support

 Strong_Sell       0.31      0.49      0.38      1580
        Hold       0.83      0.69      0.75     10891
  Strong_Buy       0.30      0.49      0.37      1596

    accuracy                           0.64     14067
   macro avg       0.48      0.56      0.50     14067
weighted avg       0.71      0.64      0.67     14067

혼동행렬:
             Sell   Hold    Buy
Strong_Sell   782   746    52
Hold        1655  7490  1746
Strong_Buy    57   757   782

피쳐 중요도 Top 20:
  19. Stoch_5_overbought            : 0.105636
   2. Stoch_6_K_above_D             : 0.076456
   1. Stoch_14_K_above_D            : 0.055122
   3. CCI_4_overbought              : 

# 과적합 및 피처성능분석

In [103]:
# =============================================================================
# 셀 9: 종합적인 모델 진단 및 개선 방안 제시 (개선된 버전)
# =============================================================================

def run_model_diagnostics(evaluation_results, feature_importance, X_train_val, y_train_val, y_test, feature_columns, data_dir):
    """
    모델 성능에 대한 종합적인 진단을 수행하고 개선 방안을 제시합니다.
    """
    print("="*60)
    print("🚀 종합적인 모델 진단 시작")
    print("="*60)

    # 1. 데이터 누수 및 시간적 순서 검증
    print("\n[1. 데이터 누수 및 시간적 순서 검증]")
    print("-"*40)
    future_leaks = [col for col in feature_columns if 'Future' in col and col != 'Future_Label']
    if future_leaks:
        print(f"  ⚠️ 경고: Future 관련 피처가 포함됨 (누수 의심): {future_leaks}")
    else:
        print("  ✅ Future 관련 피처 없음 (데이터 누수 방지 양호)")

    train_df = pd.read_csv(os.path.join(data_dir, 'train_data.csv'))
    test_df = pd.read_csv(os.path.join(data_dir, 'test_data.csv'))
    if 'Date' in train_df.columns and 'Date' in test_df.columns:
        train_max_date = pd.to_datetime(train_df['Date']).max()
        test_min_date = pd.to_datetime(test_df['Date']).min()
        if test_min_date <= train_max_date:
            print(f"  ⚠️ 경고: Train/Test 시간 중첩 발견! (Train Max: {train_max_date}, Test Min: {test_min_date})")
        else:
            print(f"  ✅ 시간적 분할 올바름 (Train: ~{train_max_date.date()}, Test: {test_min_date.date()}~)")

    # 2. 과적합 분석
    print("\n[2. 과적합 분석]")
    print("-"*40)
    overfitting_gap = evaluation_results.get('overfitting_gap', 0)
    print(f"  - Train/Test F1-Score 차이: {overfitting_gap:.4f}")
    # if overfitting_gap > 0.05:
    #     print("  - ⚠️ 평가: 과적합 가능성이 높습니다. 정규화 파라미터(alpha, lambda)를 높이거나, max_depth를 줄여보세요.")
    # else:
    #     print("  - ✅ 평가: 과적합 수준이 안정적입니다.")

    # 3. 클래스 불균형 및 가중치 효과성 분석
    print("\n[3. 클래스 불균형 및 가중치 효과성 분석]")
    print("-"*40)
    class_ratio = y_train_val.value_counts().min() / y_train_val.value_counts().max()
    print(f"  - 클래스 균형도: {class_ratio:.3f} (1.0에 가까울수록 균형)")

    actual_dist = y_test.value_counts(normalize=True).sort_index() * 100
    pred_dist = pd.Series(evaluation_results['test_predictions']).value_counts(normalize=True).sort_index() * 100
    print("  - 실제 vs 예측 분포 비교:")
    for label, name in {0: 'Sell', 1: 'Hold', 2: 'Buy'}.items():
        print(f"    - {name:5s}: 실제 {actual_dist.get(label, 0):.1f}% vs 예측 {pred_dist.get(label, 0):.1f}%")

    naive_f1 = f1_score(y_test, np.ones_like(y_test), average='macro')
    actual_f1 = evaluation_results['test_f1']
    if actual_f1 - naive_f1 < 0.05:
         print("  - ⚠️ 평가: 모델이 Hold로만 예측하는 Naive 모델 대비 성능 향상이 미미합니다. 클래스 가중치나 샘플링 기법(SMOTE 등)을 재검토하세요.")
    else:
         print("  - ✅ 평가: 클래스 가중치가 효과적으로 작동하여 Naive 모델 대비 성능이 개선되었습니다.")

    # 4. 피쳐 중요도 및 다중공선성 분석
    print("\n[4. 피쳐 중요도 및 다중공선성 분석]")
    print("-"*40)
    zero_importance_count = (feature_importance['importance'] == 0).sum()
    print(f"  - 중요도 0인 피쳐 수: {zero_importance_count}개")
    if zero_importance_count > 0:
        print("  - 💡 제안: 중요도가 0인 피쳐는 모델 성능에 기여하지 않으므로 제거를 고려해 볼 수 있습니다.")

    top_features = feature_importance.head(30)['feature'].tolist()
    corr_matrix = X_train_val[top_features].corr().abs()
    high_corr_pairs = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)).stack().reset_index()
    high_corr_pairs.columns = ['Feature 1', 'Feature 2', 'Correlation']
    high_corr_pairs = high_corr_pairs[high_corr_pairs['Correlation'] > 0.9]
    if not high_corr_pairs.empty:
        print(f"  - ⚠️ 경고: 상위 피쳐 간 높은 상관관계({len(high_corr_pairs)}쌍)가 발견되었습니다. 다중공선성 문제가 있을 수 있습니다.")
        print(high_corr_pairs.head())
    else:
        print("  - ✅ 평가: 주요 피쳐 간 높은 다중공선성 문제는 발견되지 않았습니다.")

    # 5. Future_Label과의 상관관계 분석 (새로 추가)
    print("\n[5. Future_Label과의 상관관계 분석]")
    print("-"*40)
    try:
        # 전체 데이터에서 Future_Label과의 상관관계 계산
        full_train_df = pd.concat([
            pd.DataFrame(X_train_val, columns=feature_columns),
            pd.DataFrame({'Future_Label': y_train_val})
        ], axis=1)
        
        # Future_Label과 각 피쳐 간의 상관관계 계산
        target_correlations = full_train_df[feature_columns].corrwith(full_train_df['Future_Label']).abs()
        target_correlations = target_correlations.dropna().sort_values(ascending=False)
        
        print(f"  - Future_Label과 상관관계가 높은 상위 15개 피쳐:")
        for i, (feature, corr) in enumerate(target_correlations.head(30).items(), 1):
            print(f"    {i:2d}. {feature:<30}: {corr:.4f}")
        
        # 상관관계가 매우 낮은 피쳐들
        low_corr_features = target_correlations[target_correlations < 0.01]
        if len(low_corr_features) > 0:
            print(f"\n  - ⚠️ Future_Label과 상관관계가 매우 낮은 피쳐 ({len(low_corr_features)}개):")
            print(f"    💡 제거 후보: {list(low_corr_features.head(10).index)}")
        else:
            print("  - ✅ 모든 피쳐가 Future_Label과 적절한 상관관계를 보입니다.")
            
    except Exception as e:
        print(f"  - ❌ 상관관계 분석 중 오류: {e}")

    # 6. 다중공선성 Top 30 상세 분석 (새로 추가)
    print("\n[6. 다중공선성 Top 30 상세 분석]")
    print("-"*40)
    try:
        # 상위 30개 피쳐 간 상관관계 매트릭스
        top_30_features = feature_importance.head(30)['feature'].tolist()
        corr_matrix_30 = X_train_val[top_30_features].corr().abs()
        
        # 상관관계가 높은 모든 쌍 찾기 (0.7 이상)
        high_corr_pairs_detailed = []
        for i in range(len(corr_matrix_30.columns)):
            for j in range(i+1, len(corr_matrix_30.columns)):
                corr_val = corr_matrix_30.iloc[i, j]
                if corr_val > 0.7:  # 0.7 이상 상관관계
                    high_corr_pairs_detailed.append({
                        'Feature_1': corr_matrix_30.columns[i],
                        'Feature_2': corr_matrix_30.columns[j],
                        'Correlation': corr_val
                    })
        
        if high_corr_pairs_detailed:
            high_corr_df = pd.DataFrame(high_corr_pairs_detailed).sort_values('Correlation', ascending=False)
            print(f"  - 상위 30개 피쳐 중 높은 상관관계(>0.7) 쌍: {len(high_corr_df)}개")
            print("  - Top 10 다중공선성 쌍:")
            for i, row in high_corr_df.head(10).iterrows():
                print(f"    {row['Feature_1']:<25} ↔ {row['Feature_2']:<25}: {row['Correlation']:.4f}")
            
            # 제거 후보 추천
            feature_corr_count = {}
            for _, row in high_corr_df.iterrows():
                feature_corr_count[row['Feature_1']] = feature_corr_count.get(row['Feature_1'], 0) + 1
                feature_corr_count[row['Feature_2']] = feature_corr_count.get(row['Feature_2'], 0) + 1
            
            if feature_corr_count:
                most_correlated = sorted(feature_corr_count.items(), key=lambda x: x[1], reverse=True)[:5]
                print(f"\n  - 💡 다중공선성 제거 후보 (다른 피쳐와 높은 상관관계가 많은 순):")
                for feature, count in most_correlated:
                    print(f"    • {feature}: {count}개 피쳐와 높은 상관관계")
        else:
            print("  - ✅ 상위 30개 피쳐 간 높은 다중공선성(>0.7) 문제 없음")
            
        # 평균 상관관계 계산
        upper_triangle = corr_matrix_30.where(np.triu(np.ones(corr_matrix_30.shape), k=1).astype(bool))
        avg_correlation = upper_triangle.stack().mean()
        print(f"  - 상위 30개 피쳐 간 평균 상관관계: {avg_correlation:.4f}")
        
    except Exception as e:
        print(f"  - ❌ 다중공선성 분석 중 오류: {e}")

    print("\n" + "="*60)
    print("✅ 모델 진단 완료!")
    print("="*60)

# --- 진단 실행 ---
# 이전에 계산된 evaluation_results와 feature_importance 변수가 필요합니다.
run_model_diagnostics(
    evaluation_results=evaluation_results,
    feature_importance=feature_importance,
    X_train_val=X_train_val,
    y_train_val=y_train_val,
    y_test=y_test,
    feature_columns=feature_columns,
    data_dir=data_dir
)

🚀 종합적인 모델 진단 시작

[1. 데이터 누수 및 시간적 순서 검증]
----------------------------------------
  ✅ Future 관련 피처 없음 (데이터 누수 방지 양호)
  ⚠️ 경고: Train/Test 시간 중첩 발견! (Train Max: 2025-08-15 00:00:00, Test Min: 2024-02-16 00:00:00)

[2. 과적합 분석]
----------------------------------------
  - Train/Test F1-Score 차이: 0.3010

[3. 클래스 불균형 및 가중치 효과성 분석]
----------------------------------------
  - 클래스 균형도: 0.166 (1.0에 가까울수록 균형)
  - 실제 vs 예측 분포 비교:
    - Sell : 실제 11.2% vs 예측 17.7%
    - Hold : 실제 77.4% vs 예측 63.9%
    - Buy  : 실제 11.3% vs 예측 18.3%
  - ✅ 평가: 클래스 가중치가 효과적으로 작동하여 Naive 모델 대비 성능이 개선되었습니다.

[4. 피쳐 중요도 및 다중공선성 분석]
----------------------------------------
  - 중요도 0인 피쳐 수: 0개
  - ✅ 평가: 주요 피쳐 간 높은 다중공선성 문제는 발견되지 않았습니다.

[5. Future_Label과의 상관관계 분석]
----------------------------------------
  - Future_Label과 상관관계가 높은 상위 15개 피쳐:
     1. Stoch_K_3                     : 0.3515
     2. CCI_3                         : 0.3270
     3. Stoch_14_K_above_D            : 0.2809
     4. Stoch_6_K_above_D             : 0.2

# 백테스팅용 예측 데이터 생성

In [102]:
# =============================================================================
# 셀 10: Test Set 백테스팅용 예측 데이터 생성 (70:15:15 분할)
# =============================================================================

import pandas as pd
import numpy as np
import os
import joblib
from sklearn.metrics import accuracy_score, f1_score, classification_report

# 경로 설정
data_dir = "/workspace/AI모델/projects/coin/data/v01/crypto_xgboost"  # 전처리된 데이터 경로
output_dir = "/workspace/AI모델/projects/coin/models/v01/crypto_xgboost"  # 모델 저장 경로

print("🎯 Test Set 백테스팅: 15% Test 데이터로 예측")
print("=" * 60)

# =============================================================================
# 1. 모델 및 피처 로드
# =============================================================================
print("모델 및 피처 정보 로드 중...")

# 학습된 모델 로드
model_path = os.path.join(output_dir, 'crypto_xgboost_final_model.pkl')
final_model = joblib.load(model_path)
print(f"✅ 모델 로드 완료")

# 피처 컬럼 로드
features_path = os.path.join(output_dir, 'required_features.txt')
if not os.path.exists(features_path):
    # training_features.txt로 시도
    features_path = os.path.join(data_dir, 'training_features.txt')

with open(features_path, 'r') as f:
    feature_columns = [line.strip() for line in f.readlines()]
print(f"✅ 피처 컬럼 로드 완료: {len(feature_columns)}개")

# =============================================================================
# 2. Test Set 데이터 로드 (70:15:15 분할의 마지막 15%)
# =============================================================================
print(f"\n🚀 Test Set 데이터 로드 중...")

# Test 데이터 로드
test_data_path = os.path.join(data_dir, 'test_data.csv')

if not os.path.exists(test_data_path):
    print(f"❌ Test 데이터가 없습니다: {test_data_path}")
    print(f"먼저 B00_00을 실행해주세요!")
    raise FileNotFoundError(f"Test 데이터가 없습니다")

test_data_full = pd.read_csv(test_data_path)
print(f"✅ Test 데이터 로드 완료: {test_data_full.shape}")
print(f"📅 기간: 전체 데이터의 마지막 15% (85~100%)")
print(f"🎯 종목 수: {test_data_full['Symbol'].nunique()}")
print(f"📋 종목 리스트: {test_data_full['Symbol'].unique()}")

# Date 컬럼 처리
if 'Date' in test_data_full.columns:
    test_data_full['Date'] = pd.to_datetime(test_data_full['Date'])
    print(f"📆 데이터 기간: {test_data_full['Date'].min().date()} ~ {test_data_full['Date'].max().date()}")

# =============================================================================
# 3. 데이터 분리 검증 (중요!)
# =============================================================================
print(f"\n🔍 데이터 분리 검증:")

try:
    # 훈련+검증 데이터 확인
    train_data = pd.read_csv(os.path.join(data_dir, 'train_data.csv'))
    val_data = pd.read_csv(os.path.join(data_dir, 'val_data.csv')) if os.path.exists(os.path.join(data_dir, 'val_data.csv')) else pd.DataFrame()
    
    # 훈련+검증 데이터 결합
    if len(val_data) > 0:
        train_val_data = pd.concat([train_data, val_data], ignore_index=True)
    else:
        train_val_data = train_data
        
    train_val_data['Date'] = pd.to_datetime(train_val_data['Date'])
    
    train_val_max_date = train_val_data['Date'].max()
    test_min_date = test_data_full['Date'].min()
    
    if test_min_date > train_val_max_date:
        gap_days = (test_min_date - train_val_max_date).days
        print(f"✅ 완벽한 시간적 분리!")
        print(f"   📚 훈련+검증 데이터 종료: {train_val_max_date.date()}")
        print(f"   🎯 테스트 시작: {test_min_date.date()}")
        print(f"   ⏰ 시간 간격: {gap_days}일")
    else:
        print(f"❌ 시간적 중복 발견!")
        print(f"   훈련+검증 종료: {train_val_max_date.date()}")
        print(f"   테스트 시작: {test_min_date.date()}")
        print(f"   ⚠️ 데이터 분할에 문제가 있을 수 있습니다!")
        
except Exception as e:
    print(f"⚠️ 검증 중 오류: {e}")

# =============================================================================
# 4. Test Set으로 예측 실행
# =============================================================================
print(f"\n🤖 Test Set으로 예측 실행 중...")

# 누락된 피처 확인 및 처리
missing_features = [col for col in feature_columns if col not in test_data_full.columns]
if missing_features:
    print(f"⚠️ 누락된 피처: {missing_features}")
    for feature in missing_features:
        test_data_full[feature] = 0
        print(f"   {feature} → 0으로 채움")

# 피처 데이터 준비
X_test = test_data_full[feature_columns].fillna(0)
X_test = X_test.replace([np.inf, -np.inf], 0)
print(f"🔧 테스트용 피처 데이터: {X_test.shape}")

# 모델로 예측 실행
predictions = final_model.predict(X_test)
prediction_probabilities = final_model.predict_proba(X_test)

print(f"🎉 예측 완료!")
print(f"📊 예측 결과 분포:")
unique, counts = np.unique(predictions, return_counts=True)
for label, count in zip(unique, counts):
    label_name = {0: 'Sell', 1: 'Hold', 2: 'Buy'}[int(label)]
    pct = count / len(predictions) * 100
    print(f"  {int(label)} ({label_name}): {count:,}개 ({pct:.1f}%)")

# =============================================================================
# 5. 예측 결과를 테스트 데이터에 추가
# =============================================================================
print(f"\n📝 예측 결과 추가 중...")

# 예측 라벨 및 확률 추가
test_data_full['Predicted_Label'] = predictions
test_data_full['Pred_Prob_Sell'] = prediction_probabilities[:, 0]
test_data_full['Pred_Prob_Hold'] = prediction_probabilities[:, 1]
test_data_full['Pred_Prob_Buy'] = prediction_probabilities[:, 2]

# 예측 액션을 텍스트로 변환
label_map = {0: 'Sell', 1: 'Hold', 2: 'Buy'}
test_data_full['Predicted_Action'] = test_data_full['Predicted_Label'].map(label_map)

print(f"✅ 예측 결과가 추가된 데이터: {test_data_full.shape}")

# =============================================================================
# 6. 성능 검증 (실제 라벨과 비교)
# =============================================================================
if 'Future_Label' in test_data_full.columns:
    print(f"\n📈 Test Set 성능 검증:")
    
    actual_labels = test_data_full['Future_Label'].fillna(1).astype(int)  # NaN을 Hold로 처리
    valid_mask = actual_labels.isin([0, 1, 2])
    
    if valid_mask.sum() > 0:
        actual_valid = actual_labels[valid_mask]
        pred_valid = predictions[valid_mask]
        
        test_accuracy = accuracy_score(actual_valid, pred_valid)
        test_f1 = f1_score(actual_valid, pred_valid, average='macro')
        
        print(f"  🎯 정확도: {test_accuracy:.4f}")
        print(f"  📊 F1 Score: {test_f1:.4f}")
        print(f"  📝 유효 샘플: {len(actual_valid):,}개")
        
        # 실제 vs 예측 분포 비교
        print(f"\n📊 실제 vs 예측 라벨 분포:")
        actual_dist = pd.Series(actual_valid).value_counts(normalize=True).sort_index() * 100
        pred_dist = pd.Series(pred_valid).value_counts(normalize=True).sort_index() * 100
        
        for label, name in {0: 'Sell', 1: 'Hold', 2: 'Buy'}.items():
            actual_pct = actual_dist.get(label, 0)
            pred_pct = pred_dist.get(label, 0)
            print(f"  {name}: 실제 {actual_pct:.1f}% vs 예측 {pred_pct:.1f}%")
        
        print(f"\n📋 상세 분류 보고서:")
        print(classification_report(actual_valid, pred_valid, 
                                   target_names=['Sell', 'Hold', 'Buy'], digits=4))
    else:
        print(f"  ⚠️ 유효한 실제 라벨이 없습니다.")
else:
    print(f"\n⚠️ Future_Label이 없어 성능 검증을 건너뜁니다.")

# =============================================================================
# 7. 결과 저장
# =============================================================================
print(f"\n💾 백테스팅 결과 저장 중...")

# 완전한 테스트 데이터 저장
complete_test_path = os.path.join(data_dir, 'test_data_with_predictions.csv')
test_data_full.to_csv(complete_test_path, index=False)
print(f"✅ 예측이 포함된 테스트 데이터: {complete_test_path}")

# 백테스팅용 핵심 데이터만 추출
essential_columns = ['Date', 'Symbol', 'Close', 'Predicted_Label', 
                    'Predicted_Action', 'Pred_Prob_Sell', 
                    'Pred_Prob_Hold', 'Pred_Prob_Buy']

# 원본 OHLCV 데이터도 포함 (백테스팅에 필요)
backtest_columns = ['Date', 'Symbol', 'Open', 'High', 'Low', 'Close', 'Volume',
                   'Predicted_Label', 'Predicted_Action', 'Pred_Prob_Sell', 
                   'Pred_Prob_Hold', 'Pred_Prob_Buy']

# 컬럼 존재 여부 확인
available_backtest = [col for col in backtest_columns if col in test_data_full.columns]
if 'Future_Label' in test_data_full.columns:
    available_backtest.append('Future_Label')  # 성능 검증용으로 포함

backtest_essential = test_data_full[available_backtest].copy()

# 시간순 정렬
if 'Date' in backtest_essential.columns:
    backtest_essential = backtest_essential.sort_values(['Symbol', 'Date']).reset_index(drop=True)

print(f"📊 백테스팅 핵심 데이터: {backtest_essential.shape}")
print(f"🔍 포함된 컬럼: {list(backtest_essential.columns)}")
print(f"📝 샘플 데이터:")
print(backtest_essential.head(5))

# 백테스팅 핵심 데이터 저장
essential_test_path = os.path.join(data_dir, 'test_backtest_essential.csv')
backtest_essential.to_csv(essential_test_path, index=False)
print(f"✅ 백테스팅 핵심 데이터: {essential_test_path}")

# =============================================================================
# 8. 종목별 백테스팅 데이터 분리 저장
# =============================================================================
print(f"\n📁 종목별 백테스팅 데이터 분리 저장...")

final_backtest_dir = os.path.join(data_dir, 'final_backtest_data')
os.makedirs(final_backtest_dir, exist_ok=True)

for symbol in test_data_full['Symbol'].unique():
    symbol_data = test_data_full[test_data_full['Symbol'] == symbol].copy()
    if 'Date' in symbol_data.columns:
        symbol_data = symbol_data.sort_values('Date').reset_index(drop=True)
    
    # 백테스팅에 필요한 컬럼만 선택
    symbol_backtest = symbol_data[available_backtest].copy()
    
    output_file = os.path.join(final_backtest_dir, f'{symbol}_pure_backtest.csv')
    symbol_backtest.to_csv(output_file, index=False)
    
    print(f"  📄 {symbol}: {len(symbol_backtest):,}개 → {output_file}")

print(f"\n🎉 Test Set 백테스팅 완료!")
print(f"\n📋 생성된 파일들:")
print(f"  1. {complete_test_path} (전체 테스트 데이터)")
print(f"  2. {essential_test_path} (백테스팅 핵심 데이터)")
print(f"  3. {final_backtest_dir}/ (종목별 백테스팅 파일들)")

print(f"\n🎯 Test Set (15%) 데이터로 백테스팅 준비 완료!")
print(f"✅ 모델이 학습하지 않은 15% 데이터로 성능 측정 가능")

🎯 Test Set 백테스팅: 15% Test 데이터로 예측
모델 및 피처 정보 로드 중...
✅ 모델 로드 완료
✅ 피처 컬럼 로드 완료: 28개

🚀 Test Set 데이터 로드 중...
✅ Test 데이터 로드 완료: (14067, 60)
📅 기간: 전체 데이터의 마지막 15% (85~100%)
🎯 종목 수: 50
📋 종목 리스트: ['AAVE' 'ADA' 'AETHWETH' 'ALGO' 'AVAX' 'BCH' 'BGB' 'BNB' 'BTCB' 'BTC'
 'CBBTC32994' 'CRO' 'DAI' 'DOGE' 'DOT' 'ENA' 'ETH' 'HBAR' 'HYPE32196'
 'JITOSOL' 'LEO' 'LINK' 'LTC' 'MNT27075' 'NEAR' 'OKB' 'PEPE24478' 'SEI'
 'SHIB' 'SOL' 'STETH' 'SUI20947' 'SUSDE' 'TAO22974' 'TON11419' 'TRX'
 'UNI7083' 'USDC' 'USDE29470' 'USDS33039' 'USDT' 'WBETH' 'WBTC' 'WEETH'
 'WETH' 'WSTETH' 'WTRX' 'XLM' 'XMR' 'XRP']
📆 데이터 기간: 2024-02-16 ~ 2025-09-26

🔍 데이터 분리 검증:
❌ 시간적 중복 발견!
   훈련+검증 종료: 2025-09-05
   테스트 시작: 2024-02-16
   ⚠️ 데이터 분할에 문제가 있을 수 있습니다!

🤖 Test Set으로 예측 실행 중...
🔧 테스트용 피처 데이터: (14067, 28)
🎉 예측 완료!
📊 예측 결과 분포:
  0 (Sell): 2,494개 (17.7%)
  1 (Hold): 8,993개 (63.9%)
  2 (Buy): 2,580개 (18.3%)

📝 예측 결과 추가 중...
✅ 예측 결과가 추가된 데이터: (14067, 65)

📈 Test Set 성능 검증:
  🎯 정확도: 0.6436
  📊 F1 Score: 0.5039
  📝 유효 샘플: 14,067개

📊 